In [ ]:
pip install ucimlrepo

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

# metadata
print(cdc_diabetes_health_indicators.metadata)

# variable information
print(cdc_diabetes_health_indicators.variables)


{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # ou outras transformações que precise
from sklearn.metrics import accuracy_score  # ou outra métrica de avaliação que você for utilizar
from sklearn.pipeline import Pipeline

In [ ]:
cdc_diabetes_health_indicators.metadata

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

print(y_train.value_counts())
print(y_test.value_counts())
print(X_test.value_counts())
print(X_train.value_counts())

Diabetes_binary
0                  174667
1                   28277
Name: count, dtype: int64
Diabetes_binary
0                  43667
1                   7069
Name: count, dtype: int64
HighBP  HighChol  CholCheck  BMI  Smoker  Stroke  HeartDiseaseorAttack  PhysActivity  Fruits  Veggies  HvyAlcoholConsump  AnyHealthcare  NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  Age  Education  Income
0       0         1          23   0       0       0                     1             1       1        0                  1              0            1        0         0         0         0    7    6          8         13
                             24   0       0       0                     1             1       1        0                  1              0            1        0         0         0         1    7    6          8         11
                             23   0       0       0                     1             1       1        0                  1              0            1

In [ ]:
X_train_split, X_validacao, y_train_split, y_validacao = train_test_split(X_train, y_train,test_size=0.2, random_state=42, stratify=y_train)


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']  # Lista de colunas numéricas
categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']  # Lista de colunas categóricas
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('passthrough', 'passthrough', binary_columns),
    ]
)

def objective(trial):
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('rf', RandomForestClassifier(random_state=42))
    ])


    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    #Parâmetros do Random Forest no pipeline
    pipeline.set_params(rf__n_estimators=n_estimators,
                        rf__max_depth=max_depth,
                        rf__min_samples_split=min_samples_split,
                        rf__min_samples_leaf=min_samples_leaf,
                        rf__max_features=max_features,
                        rf__criterion=criterion)

    # Treinamento
    pipeline.fit(X_train_split, y_train_split)

    # Previsão e avaliação
    y_pred = pipeline.predict(X_validacao)
    score = accuracy_score(y_validacao, y_pred)
    return score


n_trials = 100 # or any desired number of trials
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_trials) # Pass n_trials as an argument


# Mostra os melhores parâmetros encontrados
print("Melhores parâmetros: ", study.best_params)

[I 2024-11-14 20:06:50,978] A new study created in memory with name: no-name-5dd207c1-8ded-4176-817c-ca89cdeff109
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[I 2024-11-14 20:10:05,125] Trial 0 finished with value: 0.8650126881667447 and parameters: {'n_estimators': 237, 'max_depth': 33, 'min_samples_split': 13, 'min_samples_leaf': 11, 'max_features': None, 'criterion': 'gini'}. Best is trial 0 with value: 0.8650126881667447.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
[I 2024-11-14 20:10:25,028] Trial 1 finished with value: 0.8665155584025228 and para

Melhores parâmetros:  {'n_estimators': 323, 'max_depth': 41, 'min_samples_split': 19, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'criterion': 'gini'}


In [ ]:
# Treinar o modelo e trazer as métricas
# classification_report
# Melhores parâmetros:  {'n_estimators': 323, 'max_depth': 41, 'min_samples_split': 19, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'criterion': 'gini'}


In [ ]:
from sklearn.metrics import classification_report

best_params = study.best_params
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(class_weight='balanced', random_state=42, **best_params))
])

# Fit on the original training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= 0.3).astype(int)

# Avaliar o modelo com classification_report
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.96      0.64      0.77     43667
           1       0.27      0.85      0.41      7069

    accuracy                           0.67     50736
   macro avg       0.62      0.74      0.59     50736
weighted avg       0.87      0.67      0.72     50736

